In [23]:
import sys
import os

notebook_path = os.getcwd() 
parent_dir = os.path.dirname(notebook_path)
project_root = os.path.dirname(parent_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [25]:
import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateBayesianKalmanNet import StateBayesianKalmanNet
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [27]:
state_dim_2d = 2
obs_dim_2d = 2

F_base_2d = torch.tensor([[1.0, 1.0], 
                          [0.0, 1.0]])

svd_F = torch.linalg.svd(F_base_2d)
F_true_2d = F_base_2d / svd_F.S[0]

H_true_2d = torch.eye(obs_dim_2d)

Q_true_2d = torch.eye(state_dim_2d) * 0.5 # Šum procesu
R_true_2d = torch.eye(obs_dim_2d) * 0.1 # Šum měření

# Počáteční podmínky
Ex0_true_2d = torch.tensor([[1.0], [0.0]])
P0_true_2d = torch.eye(state_dim_2d) * 1.5
F_model_2d = F_true_2d
H_model_2d = H_true_2d
Q_model_2d = torch.eye(state_dim_2d) * 0.05
R_model_2d = R_true_2d*4.0
Ex0_model_2d = torch.tensor([[0.5], [0.5]])
P0_model_2d = torch.eye(state_dim_2d) * 1.0

print("\nInicializuji 2D Linear_Canonical systém (replikace autorů)...")
sys_true = Systems.DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_true_2d, P0=P0_true_2d,
    Q=Q_true_2d, R=R_true_2d,
    F=F_true_2d, H=H_true_2d,
    device=device
)
sys_model = Systems.DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_model_2d, P0=P0_model_2d,
    Q=Q_model_2d, R=R_model_2d,
    F=F_model_2d, H=H_model_2d,
    device=device
)
print("... 2D systém inicializován.")


Inicializuji 2D Linear_Canonical systém (replikace autorů)...
... 2D systém inicializován.


In [28]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 100      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 500   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 200    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 8         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [31]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 200
NUM_TEST_TRAJ = 100
J_SAMPLES_TEST = 25

# ==============================================================================
# 2. PŘÍPRAVA DAT
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")



sys_model = Systems.DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_model_2d, P0=P0_model_2d,
    Q=Q_model_2d, R=R_model_2d,
    F=F_true_2d, H=H_true_2d,
    device=device
)


# ==============================================================================
# 3. INICIALIZACE VŠECH FILTRŮ PRO POROVNÁNÍ
# ==============================================================================
ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
akf_mismatched = Filters.AdaptiveKalmanFilter(sys_model,mdm_L=3,mdm_version=2)
kf_ideal = Filters.KalmanFilter(sys_true)
print("Všechny model-based filtry (EKF, UKF, akf) inicializovány.")


# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
# Seznamy pro ukládání výsledků z každé trajektorie
all_x_true_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
all_x_hat_classic_knet_cpu = []
all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
all_x_hat_ukf_mismatched_cpu, all_P_hat_ukf_mismatched_cpu = [], []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
all_x_hat_akf_mismatched_cpu, all_P_hat_akf_mismatched_cpu = [], []
all_x_hat_kf_ideal_cpu, all_P_hat_kf_ideal_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")


with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
      # --- D. EKF (nepřesný a ideální) ---
        ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)

        full_x_hat_ekf_m = ekf_m_res['x_filtered'] # Výsledek je již kompletní trajektorie
        full_P_hat_ekf_m = ekf_m_res['P_filtered'] # To samé pro kovarianci

        ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        full_x_hat_ekf_i = ekf_i_res['x_filtered']
        full_P_hat_ekf_i = ekf_i_res['P_filtered']

        # --- E. UKF (nepřesný a ideální) ---
        ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)

        full_x_hat_ukf_m = ukf_m_res['x_filtered']
        full_P_hat_ukf_m = ukf_m_res['P_filtered']

        ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        kf_i_res = kf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        full_x_hat_kf_i = kf_i_res['x_filtered']
        full_P_hat_kf_i = kf_i_res['P_filtered']

        # --- F. Adaptivní EKF (nepřesný) ---
        akf_m_res,_,_ = akf_mismatched.process_sequence_adaptively(y_test_seq_gpu)
        full_x_hat_akf_m = akf_m_res['x_filtered']
        full_P_hat_akf_m = akf_m_res['P_filtered']
        # --- G. Uložení všech výsledků na CPU ---
        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_m.cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
        all_x_hat_ukf_mismatched_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_mismatched_cpu.append(full_P_hat_ukf_m.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())
        all_x_hat_kf_ideal_cpu.append(full_x_hat_kf_i.cpu()); all_P_hat_kf_ideal_cpu.append(full_P_hat_kf_i.cpu())
        all_x_hat_akf_mismatched_cpu.append(full_x_hat_akf_m.cpu()); all_P_hat_akf_mismatched_cpu.append(full_P_hat_akf_m.cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
# Seznamy pro sběr metrik
mse_bkn, anees_bkn = [], []; mse_classic_knet = []; mse_knetR, anees_knetR = [], []
mse_ekf_mis, anees_ekf_mis = [], []; mse_ekf_ideal, anees_ekf_ideal = [], []
mse_ukf_mis, anees_ukf_mis = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []
mse_akf_mis, anees_akf_mis = [], []; mse_kf_ideal, anees_kf_ideal = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        def get_metrics(x_hat, P_hat):
            # Délka pravdivé trajektorie
            true_len = x_true.shape[0]
            
            # Ořízneme odhady, aby měly stejnou délku jako pravdivá data.
            # Bereme prvních `true_len` prvků (tj. pro t=0 až t=299).
            x_hat_aligned = x_hat[:true_len]
            P_hat_aligned = P_hat[:true_len]
            
            # Nyní mají všechny tenzory shodnou délku
            mse = F.mse_loss(x_hat_aligned, x_true).item()
            anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_aligned.unsqueeze(0), P_hat_aligned.unsqueeze(0))
            return mse, anees

        # Výpočty pro všechny modely
        mse, anees = get_metrics(all_x_hat_ekf_mismatched_cpu[i], all_P_hat_ekf_mismatched_cpu[i]); mse_ekf_mis.append(mse); anees_ekf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_mismatched_cpu[i], all_P_hat_ukf_mismatched_cpu[i]); mse_ukf_mis.append(mse); anees_ukf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_akf_mismatched_cpu[i], all_P_hat_akf_mismatched_cpu[i]); mse_akf_mis.append(mse); anees_akf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_kf_ideal_cpu[i], all_P_hat_kf_ideal_cpu[i]); mse_kf_ideal.append(mse); anees_kf_ideal.append(anees)
# Funkce pro bezpečné průměrování
def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 80)
print("-" * 80)
print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_mis):<20.4f} | {avg(anees_ekf_mis):<20.4f}")
print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_mis):<20.4f} | {avg(anees_ukf_mis):<20.4f}")
print(f"{'akf (Nepřesný model)':<35} | {avg(mse_akf_mis):<20.4f} | {avg(anees_akf_mis):<20.4f}")
print("-" * 80)
print(f"{'--- Benchmarks ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
print(f"{'KF (Ideální model)':<35} | {avg(mse_kf_ideal):<20.4f} | {avg(anees_kf_ideal):<20.4f}")
print("="*80)


Generuji 100 testovacích trajektorií o délce 200...
Generování dat dokončeno.
Všechny model-based filtry (EKF, UKF, akf) inicializovány.

Vyhodnocuji modely na 100 testovacích trajektoriích...
Spouštím MDM odhad Q a R...
MDM odhad dokončen.
Odhadnuté Q:
 [[ 0.59849652 -0.05553507]
 [-0.05553507  0.39763968]]
Odhadnuté R:
 [[-0.01716984  0.02312592]
 [ 0.02312592  0.20326748]]
Matice Q a R v Kalmanově filtru byly aktualizovány.
Dokončena trajektorie 1/100...
Spouštím MDM odhad Q a R...
MDM odhad dokončen.
Odhadnuté Q:
 [[0.44582351 0.1013438 ]
 [0.1013438  0.39756104]]
Odhadnuté R:
 [[ 0.1751392  -0.08565226]
 [-0.08565226  0.17979874]]
Matice Q a R v Kalmanově filtru byly aktualizovány.
Dokončena trajektorie 2/100...
Spouštím MDM odhad Q a R...
MDM odhad dokončen.
Odhadnuté Q:
 [[0.39221937 0.14110725]
 [0.14110725 0.40417302]]
Odhadnuté R:
 [[ 0.24304223 -0.09081398]
 [-0.09081398  0.12671886]]
Matice Q a R v Kalmanově filtru byly aktualizovány.
Dokončena trajektorie 3/100...
Spouští